![logo](https://resolvephilly.org/themes/custom/resolvephl-ci/logo.svg)

# Scraping and standardizing Pennsylvania Act 146 Individual Reports

**Author:** Julie Christie | Director of Data & Impact

**Partnering Team:** Our Kids

**Date:** March 28, 2024

## Background

Pennsylvania publishes reports on child fatalities and near fatalities as a result of child abuse. The reports include the age, sex, county, and date of the incident, as well as whether the family was known to the local department of human/family services in the 16 months preceding the incident. The detailed reports are organized online in a way that can be scraped for some, but not all of the information. The first page of the `.pdf` reports contain the rest of the necessary information.

### Goal of Analysis

Specifically, Resolve is looking to understand the frequency at which children who experience abuse that results in their death/near death are already known to the system. We are exploring these rates at the county level to understand what the statewide trend is, and whether Philadelphia exceeds that trend.

### Glossary

-   **Act 146** -- *"Act 146 of 2006 went into effect on May 8, 2007. A major provision of this law requires that the department prepare a non-identifying summary for the governor and the General Assembly of findings for each case of substantiated child abuse or neglect that has resulted in a child fatality or near fatality."*
-   **Near fatality** -- *Definition TKTK*
-   **DA De-certification** -- *This gets assigned to a report when the District Attorney determines that the incident was not a result of child abuse.*

### Data

-   [Child Fatality/Near Fatality Quarterly Reports](https://www.pa.gov/en/agencies/dhs/resources/data-reports/quarterly-summaries-child-abuse.html) --- A collection of brief summaries of fatalities/near fatalities of children due to abuse. | No metadata available

### Tools

-   [Python](python.org) -- *Base code to facilitate scraping*
-   [Pandas](https://pandas.pydata.org/) -- *More robust data anlysis*
-   [Regex](https://developers.google.com/edu/python/regular-expressions) -- *Regular Expressions, or Regex, to parse out patterns of characters*
-   [PDF Plumber](https://github.com/jsvine/pdfplumber) -- *Parse information from .pdf files*
-   [Excel](https://www.microsoft.com/en-us/microsoft-365/p/excel/cfq7ttc0hr4r?activetab=pivot:overviewtab) -- *Clean and analyze tabulated data*

### Limitations
- A "certifying pysician" makes an individual call on whether a child's death/near death is the result of abuse, meaning that human error may result in cases not being documented in these reports
- Child fatalities and near fatalities as a result of abuse are an incredibly small and extreme subset of the overall abuse that children face. This analysis does not constitute a full picture, but rather is a snapshot of what the state deemed the most egregious cases.
- These quarterly reports may not contain all instances. A previous scrape of individual reports rendered about 2,400 reports. This scrape yielded TKTK summaries.

## Cleaning

1. Download all the reports from the Pennsylvania DHS site.

### Parse single report

In [1]:
pip install pdfplumber


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


1. Import the needed libraries

In [3]:
import re #regular expressions

import pdfplumber
import pandas as pd
from collections import namedtuple

1. Create the column names for the data that you are extracting from the pdf.

In [ ]:
Line = namedtuple('Line', 'age sex fatality date cause agency perpetrator indicated_date known_to_agency')

3. Create a regular expressions function to parse out `known` and `county`

In [ ]:
known_re = re.compile(
    
)